In [1]:
import sys
sys.path.append("..")

from Conversation.serializers import *
from Conversation.converse.core import Message
from Conversation.converse.prompts import *
from Conversation.converse.tools import *

from turbochat.gptprompts import *

import numpy as np
import pandas as pd

In [2]:
def collected_health_information_entries(entries):
    df = pd.DataFrame(entries)
    df.columns = ["i_parameter_label", "parameter_type", "parameter_value"]
    return df

CONVERSE_TOOLS = Tools([
    {
        "name": "collected_health_information_entries",
        "function": collected_health_information_entries,
        "definition": EXTRACT_USER_RELATED_INFO
    }
])

In [3]:
        
context = {
    "message": "I have diabetes",
    "assistant_instructions": "",
    "history": "",
    "goals": "",
    "events": "",
    "doctors": "",
    "doctors": ""
}

message = Message(USER_PROMPT_DEFAULT, context, SYSTEM_MESSAGE, history=None, tools=CONVERSE_TOOLS)
reply, tool_calls = message.get_results()




In [4]:
reply, tool_calls

("I'm sorry to hear about your diabetes. How long have you been diagnosed with diabetes? Have you been able to manage it well? Have you had any recent medical tests or check-ups related to your diabetes? It's important to stay on top of your health when managing a condition like diabetes. Have you made any lifestyle changes to help control your blood sugar levels?\n",
 {'collected_health_information_entries': [  parameter_label     parameter_type parameter_value
   0        diabetes  medical_condition         present]})

In [12]:
if tool_calls and "collected_health_information_entries" in tool_calls:
    history = tool_calls["collected_health_information_entries"][0]

import json

print(json.dumps(history.to_dict("records"), indent=2))

[
  {
    "parameter_label": "diabetes",
    "parameter_type": "medical_condition",
    "parameter_value": "present"
  }
]
